# Regla de Bayes i aplicació a la classificació de text

En aquest pràctica introduïrem dos conceptes que no vem arribar a veure a les classes de teoria:

* la regla de Bayes [Casella & Berger 1.3.5],
* i la independència condicional

i els aplicarem per desenvolupar un classificador de text binari (dues classes).

La pràctica comença amb una mica de teoria, conté alguns exercicis de paper i llapis i finalment alguns exercicis computacionals. El codi el podeu desenvolupar en R o en Python, com us vingui millor. 

Els resultats dels exercicis els haureu d'entregar pel **dia 5/20/2020 abans de les 9:00**. Durant la classe d'aquest dia us en donaré la resolució.

Recordeu que heu d'entregar el resultat de la pràctica per parelles (amb excepció d'algun triplet en cas de ser senars). No oblideu incloure el vostre Nom i NIU a la tramesa. 

## Regla de Bayes

Donada una partició $A_1, A_2, \cdots, A_N$ d'$\Omega$, la regla de Bayes ens permet calcular la probabilitat d'un esdeveniment $A_i$ donat un altre esdeveniment $B$ com [Casella & Berger 1.3.5]:

$P(A_i|B) = \frac{P(B|A_i)P(A_i)}{\sum_{j=1}^NP(B|A_j)P(A_j)}$

Normalment anomenarem:

* $P(A_i|B)$ la probabilitat *posterior* d'$A_i$ havent observat $B$,
* $P(B|A_i)$ la *versemblança* de B condicionat a $A_i$
* $P(A_i)$ l'*a priori* d'$A_i$

La fórmula és fàcil de demostrar a partir de l'identitat $P(A|B) P(B) = P(A \cap B) = P(B|A) P(A)$.

Vegem amb un exercici d'exemple l'importància d'aquesta fórmula.

### Exercici 1

Considereu un test mèdic per una certa patologia. El test té dos resultats possibles: $\left\{+, -\right\}$. Per altra banda, una persona d'una població pot tenir o no tenir la patologia, que denotarem per $\left\{\mbox{Patologia}, \overline{\mbox{Patologia}}\right\}$.

Suposem que el test no és perfecte i per tant comet errors. Per exemple:

* $P(+ | \overline{\mbox{Patologia}}) = 0.1$ (fals positiu)
* $P(- | {\mbox{Patologia}}) = 0.05$ (fals negatiu)

Per altra banda, sabem que la prevalència de la patologia en la població és molt baixa, posem
$P({\mbox{Patologia}}) = 0.001$. 

Si ens fem el test i ens dona positiu, quina és la probabilitat de que realment sofrim la patologia?

Sol.lució:

$P(\mbox{Patologia} | +) = \frac{P(+ |\mbox{Patologia})P(\mbox{Patologia})}{P(+ |\mbox{Patologia})P(\mbox{Patologia}) + P(+ |\overline{\mbox{Patologia}})P(\overline{\mbox{Patologia}})} = \frac{0.95 \times 0.001}{0.95 \times 0.001 + 0.1 \times 0.999} = \frac{0.00095}{0.10085} \approx 0.009$

## Classificació 

La regla de Bayes es pot fer servir per dissenyar classificadors. Per exemple, considereu els esdeveniments definits sobre una població de correus electrònics:

* $A_1 = \mbox{SPAM}$: El correu és spam (no desitjat)
* $A_2 = \mbox{HAM}$: El correu no és spam (també anomenat "ham")
* $B$: L'esdeveniment corresponent a la presència o absència de $K$ certes paraules en el text del correu. Per exemple $B=W_1 \cap W_2^c\cap W_3$ es correspondria amb un correu el text del qual conté les paraules $W_1$ i $W_3$ i no conté la paraula $W_2$.

Un classificador de correus electrònics és una funció que pren el text corresponent a $B$ i retorna $\left\{\mbox{SPAM}, \mbox{HAM}\right\}$. Considerem la següent funció:

$f(B) = \left\{\begin{array}{cc}\mbox{SPAM} & \mbox{ si } P(\mbox{SPAM}|B) > P(\mbox{HAM}|B) \\\mbox{HAM} & \mbox{altrament} \end{array}\right.$


### Exercici 2

Perquè té sentit una tal proposta?

### Exercici 3

En realitat, gràcies a la regla de Bayes, podem definir el nostre classificador en funció només del text del correu. Per fer-ho aplicarem la regla de Bayes a la definició de $f(B)$ per obtenir:

$f(B) = \left\{\begin{array}{cc}\mbox{SPAM} & \mbox{ si } P(B|\mbox{SPAM})P(\mbox{SPAM}) > P(B|\mbox{HAM})P(\mbox{HAM}) \\\mbox{HAM} & \mbox{altrament} \end{array}\right.$

Anem a provar el nostre classificador. Per tal d'implementar-lo, hauriem de tenir coneixement de les quantitats $P(B|\mbox{SPAM}), P(\mbox{SPAM}), P(B|\mbox{HAM}), P(\mbox{HAM})$ per qualsevol B, que en principi no tenim. Per tant, les haurem d'**estimar** a partir de **dades d'entrenament** (*training data* en anglès).

Per començar, ens imaginarem que volem classificar els correus només en funció de la presència o absència de dues paraules: "prince", "USD". L'esdeveniment $B$ per tant ens indica si alguna, cap, o totes aquestes paraules són presents en un determinat correu. Per exemple el correu amb text "I am a Nigerian prince - send me 1000USD" es correspon amb l'esdeveniment $B=\mbox{prince} \cap \mbox{USD}$ i el text "Hey dude - you still owe me 10USD!" es correspon amb l'esdeveniment $B=\mbox{prince}^c \cap \mbox{USD}$.

Construïu un conjunt de dades d'entrenament "de joguina" amb 10 files. Us en dono dues d'exemple:

|  Etiqueta | Correu                             |  "prince" | "USD"  |
|-----------|------------------------------------|-----------|--------|
|  $\mbox{SPAM}$     | I am a Nigerian prince - send me 1000USD  |      1    |   1    |
|  $\overline{\mbox{SPAM}}$  | Hey Arnau - how have you been?     |      0    |   0    |

### Exercici 4

Amb el vostre conjunt de dades, estimeu les quantitats $P(B|\mbox{SPAM}), P(\mbox{SPAM}), P(B|\mbox{HAM}), P(\mbox{HAM})$. *Pista*: L'estimador més simple de $P(B = \mbox{prince}^c \cap \mbox{USD}^c|\mbox{SPAM})$ seria

$\hat{P}(B = \mbox{prince}^c \cap \mbox{USD}^c |\mbox{SPAM}) = \frac{\mbox{# de correus amb etiqueta SPAM que no contenen "prince" ni "USD"}}{\mbox{# de correus amb etiqueta SPAM}}$

Fantàstic! Un cop ja teniu estimat $P(B|\mbox{SPAM}), P(\mbox{SPAM}), P(B|\mbox{HAM}), P(\mbox{HAM})$ ja teniu "entrenat" el vostre classificador de Bayes, que podeu fer servir amb qualsevol correu nou, només mirant si conté aquestes dues paraules clau. 

Creieu que el vostre serà un bon classificador? (Justifiqueu la resposta)

## Classificació per Bayes naïf

El model que hem dissenyat fins ara és una mica limitat: tots ens podem imaginar varietats de correus de spam que no serien detectats pel nostre primer classificador (falsos negatius), o correus que no són spam que serien identificats com a tal (falsos positius). Això es deu en part al fet que fins ara només ens hem basat en dues paraules per prendre la nostra decisió.

Sorprenentment però, l'idea bàsica que hem fet servir dona lloc a un classificador anomenat de Bayes Naïf, que fins fa relativament poc (10-15 anys) era competitiu respecte als millors classificadors d'aprentissatge automàtic per problemes d'aquest tipus.

Considereu doncs el cas on volem construir el nostre classificador fent servir un vocabulari més ampli. El problema de fer servir un vocabulari més ampli és que haurem d'estimar $P(B|\mbox{SPAM})$ i $P(B|\mbox{HAM})$ per moltes possibles combinacions de $B$. 

### Exercici 5

Si en l'exemple de l'Exercici 4, quan feiem servir 2 paraules, hem hagut d'estimar $P(B|\mbox{SPAM})$ i $P(B|\mbox{HAM})$ per les 4 possibles combinacions de B, si en fem servir $N$, per quantes combinacions ho haurem de fer?

## La naiveté

La idea fonamental del mètode de Bayes Naïf és la utilització de la següent aproximació:

$P\left(B = \cap_{i=1}^K W_i | A\right) \approx P(W_1|A)P(W_2|A)\cdots P(W_K|A)$

La igualtat seria certa només en el cas que els esdeveniments $W_i$ fóssin independents condicionals a $A$. (Si us pica la curiositat, podeu buscar un exemple d'esdeveniments $A_1$, $A_2$ que són independents condicionals a un tercer esdeveniment $B$, però no són independents.)

Amb aquesta aproximació, enlloc d'haver d'estimar $P\left(B = \cap_{i=1}^K W_i | A\right)$ (cosa complicada com hem vist en l'Exercici 5) només haurem d'estimar $P(W_1|A), P(W_2|A), \cdots P(W_K|A)$. Per posar-ho en termes de l'exemple de l'Exercici 4, amb aquesta aproximació calculariem $\hat{P}(B = \mbox{prince}^c \cap \mbox{USD}^c |\mbox{SPAM})$ com:

$\hat{P}(B = \mbox{prince}^c \cap \mbox{USD}^c |\mbox{SPAM}) = \frac{\mbox{# de correus SPAM que no contenen "prince" }}{\mbox{# de correus SPAM}} \times \frac{\mbox{# de correus SPAM que no contenen "USD" }}{\mbox{# de correus SPAM}}$

Som-hi doncs. Per aplicar aquest concepte, ens descarregarem un conjunt de dades de classificació de correus "de veritat". 

### Exercici 6


1. Visiteu [aquesta pàgina i premeu "Donwload"](https://www.kaggle.com/venky73/spam-mails-dataset). Carregueu les dades amb pandas (o en la vostra llibreria preferida d'R), i exploreu-les: quants exemples tenim per cada classe (SPAM/HAM)? Quantes paraules hi ha en el text de cada correu? Quines són les paraules més comuns?

In [1]:
import pandas as pd

dataset = pd.read_csv('/Users/arnau.tibau/Downloads/spam_ham_dataset 2.csv')

dataset.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


2. Separeu el conjunt de dades en un conjunt d'entrenament amb les 15000 primeres files i un de test amb les files restants. *NOTA*: normalment no és bona idea separar l'entrenament del test així, en aquest cas no tindrem cap problema perquè l'ordre de les dades és aleatori.

*Pels següents exercicis, us he facilitat uns tests al final de l'anunciat perquè veieu quin és el comportament esperat de les funcions que haurem d'implementar. Us recomano fortament que per cada funció que implementeu, escrigueu una sèrie de **[tests unitaris](https://en.wikipedia.org/wiki/Unit_testing)** que comprovin que la funció es comporta com és esperat per una sèrie d'exemples.*

3. Implementeu una funció `estimate_word_frequency(data, word)` que prengui com a input el conjunt de dades d'entrenament (`data`) i una cadena de caràcters `word`, i retorni la frequència de la paraula en cada classe (és a dir, un estimador de $P(\mbox{word}|\mbox{SPAM})$ i $P(\mbox{word}|{\mbox{HAM}})$ )
4. Apliqueu aquest a funció a les paraules ["Subject:", "company", "http", "information", "enron", "gas", "deal", "meter", "forwarded"] i emmagatzemeu els resultats en un diccionary `word_frequencies` tal que per cada paraula hi teniu la seva frequència en SPAM i HAM.
5. Implementeu la funció `estimate_spam_frequency(data)` que pren com a input el conjunt de dades i retorna una estimació de $P(\mbox{SPAM})$.
6. Implementeu una funció `estimate_likelihood(text, word_frequencies)` que pren com a input el text d'un correu (`text`), i l'estimació de freqüències de cada paraula (`word_frequencies`) i retorna una estimació de les versemblances $P(B|{\mbox{SPAM}})$ i $P(B|\mbox{HAM})$ fent servir l'aproximació de Bayes naïf.
7. Implementeu la funció `naive_bayes(text, word_frequencies, spam_frequency)` que pren com a input el text d'un correu (`text`), l'estimació de la freqüència d'spam (`spam_frequency`) i l'estimació de freqüències de cada paraula (`word_frequencies`) i retorna 'SPAM' o 'HAM' en funció de la regla que hem definit a $f(B)$ (cf. "Classificació"), fent servir l'aproximació de Bayes Naïf. Per fer-ho, aprofiteu la funció `estimate_likelihood(text, word_frequencies)` que acabeu d'implementar. Us podria anar bé retornar també $P({\mbox{SPAM}}|B)$, $P({\mbox{HAM}}|B)$ per depurar la implementació.
8. Proveu el vostre classificador amb alguns exemples del conjunt d'entrenament, i alguns del conjunt de test. Què tal funciona? Classifica algun exemple correctament? 
9. [OPCIONAL] Calculeu la precisió (*accuracy*) del vostre classificador.
9. [OPCIONAL] Quin és el principal problema del vostre classificado? Què podriem fer per millorar-lo? Implementeu la millor i mireu si obteniu un guany en precisió.

In [2]:
# Solucions 3, 5, 6, 7
def estimate_word_frequency(data, word):
    word_freq = dict()
    spam_data = data.loc[data.label=='spam']
    word_freq['spam'] = spam_data.apply(lambda x: word in x.text, axis=1).sum()/spam_data.shape[0]
    not_spam_data = data.loc[data.label=='ham'] 
    word_freq['ham'] = not_spam_data.apply(lambda x: word in x.text, axis=1).sum()/not_spam_data.shape[0]
    return word_freq

def estimate_spam_frequency(data):
    return data.loc[data.label=='spam'].shape[0]/data.shape[0]

def estimate_likelihood(text, word_frequencies):
    p_b_given_spam = 1
    p_b_given_ham = 1
    words_found = 0
    for word, freqs in word_frequencies.items():
        if word in text:
            p_b_given_spam = p_b_given_spam * freqs['spam']
            p_b_given_ham = p_b_given_ham * freqs['ham']
            words_found += 1
    if words_found > 0:
        return p_b_given_spam, p_b_given_ham
    else:
        return 0., 0.
    
def naive_bayes(text, word_frequencies, spam_frequency):
    p_b_given_spam, p_b_given_ham = estimate_likelihood(text, word_frequencies)
    if p_b_given_spam*spam_frequency > p_b_given_ham*(1-spam_frequency):
        return 'spam'
    else:
        return 'ham'

In [3]:
# Comportament esperat de les funcions a implementar
example_data = pd.DataFrame({
    'label': ['spam', 'spam', 'ham', 'ham'],
    'text': [
        'A B C D',
        'E F G C',
        'A F G',
        'C D E F'
    ]
})
assert estimate_word_frequency(example_data, 'A') == {'spam': 1./2, 'ham': 1./2}
assert estimate_word_frequency(example_data, 'B') == {'spam': 1./2, 'ham': 0}
assert estimate_word_frequency(example_data, 'C') == {'spam': 1, 'ham': 1./2}


assert estimate_spam_frequency(example_data) == 1./2

word_frequencies = {
    'A': {'spam': 1./2, 'ham': 1./2},
    'B': {'spam': 1./2, 'ham': 0},
    'C': {'spam': 1, 'ham': 1./2}
}
assert estimate_likelihood('A B', word_frequencies) == (1./4, 0)
assert estimate_likelihood('A C', word_frequencies) == (1./2, 1./4)

spam_frequency = estimate_spam_frequency(example_data)
assert naive_bayes('B', word_frequencies, spam_frequency) == 'spam'
assert naive_bayes('A B C', word_frequencies, spam_frequency) == 'spam'
assert naive_bayes('F', word_frequencies, spam_frequency) == 'ham'

In [4]:
# Algunes funcions auxiliars que us podrien ser útils

def word_count(text_list, min_char=3):
    """
    Retorna el nombre de vegades que apareix cada paraula d'al menys `min_char`
    caràcters a la llista de texts `text_list`
    """
    counts = {}
    for text in text_list:
        for word in text.split():
            if len(word) < min_char:
                continue
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
            
    return sorted(counts.items(), key=lambda x: -x[1])

In [5]:
# Solucions 8, 9
training_data = dataset.head(15000)
test_data = dataset.tail(5000)

dictionary = ["Subject:", "company", "http", "information", "enron", "gas", "deal", "meter", "forwarded"] 

word_frequencies = {}
for word in dictionary:
    word_frequencies[word] = estimate_word_frequency(training_data, word)

spam_frequency = estimate_spam_frequency(example_data)

In [6]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = test_data.text.apply(lambda text: naive_bayes(text, word_frequencies, spam_frequency))
y_true = test_data.label

print(classification_report(y_true, y_pred))

print('accuracy:', accuracy_score(y_true, y_pred))

              precision    recall  f1-score   support

         ham       0.79      0.96      0.87      3552
        spam       0.78      0.39      0.52      1448

    accuracy                           0.79      5000
   macro avg       0.79      0.67      0.70      5000
weighted avg       0.79      0.79      0.77      5000

accuracy: 0.7926


In [7]:
accuracy_score(y_true, ['ham']*len(y_true))

0.7104

In [8]:
for _, row in test_data.head().iterrows():
    prediction = naive_bayes(row['text'], word_frequencies, spam_frequency)
    print(f"{row['text']} --> Predicted={prediction} True={row['label']}")

Subject: re : e mail list for class
please respond to great idea , byron !
everybody ,
i ' ve created a new moderated list at http : / / . listbot . com /
if you are interested in hearing info ( from anyone , not just me ) about the
bammel road young families class , please go to the address above to
subscribe to the list .
it ' s free .
then , if you have anything that you want to share with the class , just
e - mail it to @ listbot . com .
- ram .
- - - - - original message - - - - -
from : byron w . ellis , cfp , clu , chfc [ mailto : byronellis @ usa . net ]
sent : friday , march 30 , 2001 8 : 31 am
to : ram tackett
subject : e mail list for class
what do you think about creating and maintaining an e mail address that will
automatically send an e mail to everyone in our class ? that way anyone that
wanted to blanket the class could always do so without going through you all
the time .
byron
" make all you can , save all you can , give all you can . "
john wesley
byron w . ellis , c